# ERCOT Price and Weather Data Exploration and Visualization

This script provides comprehensive exploration and visualization of ERCOT price
and weather data to gain insights for forecasting models.

## Setup and Imports

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union, Any

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")
# %matplotlib inline

In [ ]:
from src.data.ercot_price_data import ErcotPriceData
from src.data.ercot_weather_data import ErcotWeatherData
from src.utils.preprocessing import align_time_series, create_time_features

## Data Loading

Let's load price and weather data for multiple locations in ERCOT.

In [ ]:
# Define date range - look at last 2 years of data
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)  # 2 years of data

In [ ]:
# Define price nodes and locations to explore
price_nodes = ['HB_HOUSTON', 'HB_NORTH', 'HB_SOUTH', 'HB_WEST']
locations = ['Houston', 'Dallas', 'San Antonio', 'Midland']

# Load price data for all nodes
price_data = {}
for node in price_nodes:
    price_data[node] = ErcotPriceData().load_data(
        start_date=start_date,
        end_date=end_date,
        price_node=node,
        resample_freq='H'  # Hourly data
    )
    print(f"Loaded price data for {node}: {price_data[node].shape} rows")

In [ ]:
# Load weather data for all locations
weather_data = {}
for location in locations:
    weather_data[location] = ErcotWeatherData().load_data(
        start_date=start_date,
        end_date=end_date,
        location=location,
        resample_freq='H'  # Hourly data
    )
    print(f"Loaded weather data for {location}: {weather_data[location].shape} rows")

## Basic Price Data Statistics

Let's examine the statistics of price data across different ERCOT hubs.

In [ ]:
# Create a summary DataFrame of price statistics
price_stats = pd.DataFrame()

for node in price_nodes:
    # Basic statistics
    node_stats = {
        'Mean': price_data[node]['price'].mean(),
        'Median': price_data[node]['price'].median(),
        'Std Dev': price_data[node]['price'].std(),
        'Min': price_data[node]['price'].min(),
        'Max': price_data[node]['price'].max(),
        'Skew': price_data[node]['price'].skew(),
        'Kurtosis': price_data[node]['price'].kurtosis(),
        '95th Percentile': price_data[node]['price'].quantile(0.95),
        '99th Percentile': price_data[node]['price'].quantile(0.99)
    }
    price_stats[node] = pd.Series(node_stats)

print("Price Statistics Across ERCOT Hubs")
price_stats.T

## Price Distribution Analysis

Let's visualize the distribution of electricity prices across different hubs.

In [ ]:
# Create a figure with histograms for each node
fig = plt.figure(figsize=(16, 10))

for i, node in enumerate(price_nodes):
    ax = fig.add_subplot(2, 2, i+1)
    
    # Plot histogram with KDE
    sns.histplot(price_data[node]['price'], kde=True, bins=100, ax=ax)
    
    # Add vertical lines for mean and median
    plt.axvline(price_data[node]['price'].mean(), color='r', linestyle='--', label='Mean')
    plt.axvline(price_data[node]['price'].median(), color='g', linestyle='-.', label='Median')
    
    # Set title and labels
    plt.title(f'{node} Price Distribution')
    plt.xlabel('Price ($/MWh)')
    plt.ylabel('Frequency')
    plt.legend()
    
    # Limit x-axis to better visualize the distribution (excluding extreme outliers)
    plt.xlim(0, price_data[node]['price'].quantile(0.99))

plt.tight_layout()
plt.show()

## Price Spike Analysis

Let's analyze the frequency and magnitude of price spikes.

In [ ]:
# Define thresholds for moderate and extreme price spikes
moderate_threshold = 100  # $/MWh
extreme_threshold = 300   # $/MWh

# Calculate spike statistics
spike_stats = pd.DataFrame()

for node in price_nodes:
    prices = price_data[node]['price']
    
    # Calculate spike metrics
    moderate_spikes = (prices > moderate_threshold) & (prices <= extreme_threshold)
    extreme_spikes = prices > extreme_threshold
    
    node_stats = {
        'Moderate Spikes (%)': moderate_spikes.mean() * 100,
        'Extreme Spikes (%)': extreme_spikes.mean() * 100,
        'Max Spike ($/MWh)': prices.max(),
        'Average Spike Duration (hours)': 0,  # Will calculate below
        'Longest Spike Duration (hours)': 0   # Will calculate below
    }
    
    # Calculate spike durations
    in_spike = False
    current_duration = 0
    durations = []
    
    for is_spike in (moderate_spikes | extreme_spikes):
        if is_spike:
            in_spike = True
            current_duration += 1
        elif in_spike:
            durations.append(current_duration)
            in_spike = False
            current_duration = 0
    
    # Add the last spike if we ended during one
    if current_duration > 0:
        durations.append(current_duration)
    
    # Update statistics
    if durations:
        node_stats['Average Spike Duration (hours)'] = np.mean(durations)
        node_stats['Longest Spike Duration (hours)'] = np.max(durations)
    
    spike_stats[node] = pd.Series(node_stats)

print("Price Spike Statistics Across ERCOT Hubs")
spike_stats.T

## Time Series Visualization of Prices

Let's visualize the price time series and identify patterns.

In [ ]:
# Create an interactive plot with all price nodes
fig = go.Figure()

for node in price_nodes:
    fig.add_trace(go.Scatter(
        x=price_data[node].index,
        y=price_data[node]['price'],
        mode='lines',
        name=node
    ))

# Update layout
fig.update_layout(
    title='ERCOT Hub Prices',
    xaxis_title='Date',
    yaxis_title='Price ($/MWh)',
    height=600,
    width=1000,
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# Limit y-axis to better visualize (excluding extreme outliers)
y_max = max([price_data[node]['price'].quantile(0.99) for node in price_nodes])
fig.update_yaxes(range=[0, y_max])

fig.show()

## Price Volatility Analysis

Let's analyze price volatility over time using rolling standard deviation.

In [ ]:
# Calculate rolling volatility (standard deviation) for different windows
windows = [24, 168, 720]  # 1 day, 1 week, 1 month (in hours)
window_labels = ['1-Day', '1-Week', '1-Month']

# Create a figure with volatility plots for Houston hub
fig = go.Figure()

for window, label in zip(windows, window_labels):
    volatility = price_data['HB_HOUSTON']['price'].rolling(window=window).std()
    
    fig.add_trace(go.Scatter(
        x=price_data['HB_HOUSTON'].index,
        y=volatility,
        mode='lines',
        name=f'{label} Volatility'
    ))

# Update layout
fig.update_layout(
    title='ERCOT Houston Hub Price Volatility',
    xaxis_title='Date',
    yaxis_title='Price Volatility ($/MWh)',
    height=500,
    width=1000,
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

## Seasonal and Temporal Patterns

Let's explore seasonal and temporal patterns in electricity prices.

In [ ]:
# Align price and weather data for Houston
houston_data = align_time_series(price_data['HB_HOUSTON'], weather_data['Houston'])

# Add time features
houston_data = create_time_features(houston_data)

# Group by month and hour to create a heatmap of average prices
monthly_hourly_avg = houston_data.groupby(['month', 'hour'])['price'].mean().unstack()

# Create a heatmap of hourly prices by month
plt.figure(figsize=(14, 8))
sns.heatmap(monthly_hourly_avg, cmap='viridis', annot=False, fmt='.1f')
plt.title('Average Hourly Prices by Month (Houston Hub)')
plt.xlabel('Hour of Day')
plt.ylabel('Month')
plt.show()

# Create seasonal aggregations
seasons = {
    'Winter': [12, 1, 2],
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Fall': [9, 10, 11]
}

# Create a DataFrame for seasonal hourly patterns
seasonal_hourly = pd.DataFrame()

for season, months in seasons.items():
    seasonal_data = houston_data[houston_data['month'].isin(months)]
    hourly_avg = seasonal_data.groupby('hour')['price'].mean()
    seasonal_hourly[season] = hourly_avg

# Plot seasonal hourly patterns
plt.figure(figsize=(14, 8))
for season in seasonal_hourly.columns:
    plt.plot(seasonal_hourly.index, seasonal_hourly[season], label=season, linewidth=3)

plt.title('Hourly Price Patterns by Season (Houston Hub)')
plt.xlabel('Hour of Day')
plt.ylabel('Average Price ($/MWh)')
plt.grid(True)
plt.legend()
plt.show()

## Correlation Between Hubs

Let's analyze the correlations between different ERCOT hubs.

In [ ]:
# Create a DataFrame with all hub prices
all_prices = pd.DataFrame()
for node in price_nodes:
    all_prices[node] = price_data[node]['price']

# Calculate correlation matrix
corr_matrix = all_prices.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=.5)
plt.title('Price Correlation Between ERCOT Hubs')
plt.show()

## Weather Data Exploration
 
Let's explore the weather data for different locations.

In [ ]:
# Create a summary DataFrame of weather statistics for Houston
weather_vars = ['temperature', 'wind_speed', 'solar_irradiance', 'humidity']
houston_weather = weather_data['Houston']

weather_stats = pd.DataFrame()
for var in weather_vars:
    var_stats = {
        'Mean': houston_weather[var].mean(),
        'Median': houston_weather[var].median(),
        'Std Dev': houston_weather[var].std(),
        'Min': houston_weather[var].min(),
        'Max': houston_weather[var].max()
    }
    weather_stats[var] = pd.Series(var_stats)

print("Weather Statistics for Houston")
weather_stats.T

# Plot distributions of weather variables
fig = plt.figure(figsize=(16, 10))

for i, var in enumerate(weather_vars):
    ax = fig.add_subplot(2, 2, i+1)
    
    # Plot histogram with KDE
    sns.histplot(houston_weather[var], kde=True, bins=50, ax=ax)
    
    # Add vertical lines for mean and median
    plt.axvline(houston_weather[var].mean(), color='r', linestyle='--', label='Mean')
    plt.axvline(houston_weather[var].median(), color='g', linestyle='-.', label='Median')
    
    # Set title and labels
    plt.title(f'Houston {var.capitalize()} Distribution')
    plt.xlabel(var.capitalize())
    plt.ylabel('Frequency')
    plt.legend()

plt.tight_layout()
plt.show()

## Seasonal Weather Patterns

Let's analyze seasonal patterns in weather variables.

In [ ]:
# Create monthly averages for each weather variable in Houston
monthly_weather = pd.DataFrame()

for var in weather_vars:
    monthly_weather[var] = houston_weather.groupby(houston_weather.index.month)[var].mean()

# Plot monthly weather patterns
fig = plt.figure(figsize=(16, 10))

for i, var in enumerate(weather_vars):
    ax = fig.add_subplot(2, 2, i+1)
    
    # Plot monthly average
    months = range(1, 13)
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    plt.bar(months, monthly_weather[var])
    plt.xticks(months, month_names)
    
    # Set title and labels
    plt.title(f'Monthly Average {var.capitalize()} in Houston')
    plt.xlabel('Month')
    plt.ylabel(var.capitalize())

plt.tight_layout()
plt.show()

## Price vs. Weather Analysis

Let's analyze the relationship between price and weather variables.

In [ ]:
# Use aligned data for Houston
# Create scatter plots for each weather variable vs. price
fig = plt.figure(figsize=(16, 10))

for i, var in enumerate(weather_vars):
    ax = fig.add_subplot(2, 2, i+1)
    
    # Plot scatter with trend line
    sns.regplot(x=houston_data[var], y=houston_data['price'], scatter_kws={'alpha': 0.3}, line_kws={'color': 'red'}, ax=ax)
    
    # Set title and labels
    plt.title(f'Price vs. {var.capitalize()}')
    plt.xlabel(var.capitalize())
    plt.ylabel('Price ($/MWh)')
    
    # Limit y-axis to better visualize (excluding extreme outliers)
    plt.ylim(0, houston_data['price'].quantile(0.99))

plt.tight_layout()
plt.show()

# Calculate correlation matrix for Houston price and weather
corr_matrix = houston_data[['price'] + weather_vars].corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=.5)
plt.title('Correlation: Price vs. Weather Variables (Houston)')
plt.show()

## Price vs. Weather by Season

Let's analyze how the relationship between price and weather varies by season.

In [ ]:
# Create seasonal data for Houston
seasonal_data = {}
for season, months in seasons.items():
    seasonal_data[season] = houston_data[houston_data['month'].isin(months)]

# Plot the relationship between temperature and price for each season
plt.figure(figsize=(12, 8))

for season, data in seasonal_data.items():
    plt.scatter(data['temperature'], data['price'], alpha=0.3, label=season)

plt.title('Price vs. Temperature by Season (Houston)')
plt.xlabel('Temperature (°C)')
plt.ylabel('Price ($/MWh)')
plt.ylim(0, houston_data['price'].quantile(0.99))
plt.legend()
plt.grid(True)
plt.show()

# Calculate seasonal correlations between price and weather
seasonal_corr = {}
for season, data in seasonal_data.items():
    seasonal_corr[season] = data[['price'] + weather_vars].corr()['price'].drop('price')

# Create a DataFrame with seasonal correlations
seasonal_corr_df = pd.DataFrame(seasonal_corr)
seasonal_corr_df

# Plot seasonal correlations as a bar chart
seasonal_corr_df.plot(kind='bar', figsize=(12, 6))
plt.title('Seasonal Correlation: Price vs. Weather Variables (Houston)')
plt.xlabel('Weather Variable')
plt.ylabel('Correlation with Price')
plt.grid(True)
plt.show()

## Extreme Weather Impact Analysis

Let's analyze how extreme weather conditions impact electricity prices.

In [ ]:
# Define thresholds for extreme weather conditions in Houston
extreme_conditions = {
    'High Temperature': houston_weather['temperature'] > houston_weather['temperature'].quantile(0.95),
    'Low Temperature': houston_weather['temperature'] < houston_weather['temperature'].quantile(0.05),
    'High Wind': houston_weather['wind_speed'] > houston_weather['wind_speed'].quantile(0.95),
    'Low Solar': houston_weather['solar_irradiance'] < houston_weather['solar_irradiance'].quantile(0.05),
    'High Humidity': houston_weather['humidity'] > houston_weather['humidity'].quantile(0.95)
}

# Create an aligned price series
aligned_price = align_time_series(price_data['HB_HOUSTON'], houston_weather)['price']

# Calculate average prices during extreme conditions vs. normal conditions
extreme_price_impact = pd.DataFrame(columns=['Avg Price ($)', 'Price Increase (%)', 'Volatility Increase (%)'])

normal_price_avg = aligned_price.mean()
normal_price_vol = aligned_price.std()

for condition, mask in extreme_conditions.items():
    extreme_price = aligned_price[mask]
    extreme_price_avg = extreme_price.mean()
    extreme_price_vol = extreme_price.std()
    
    price_increase = (extreme_price_avg / normal_price_avg - 1) * 100
    vol_increase = (extreme_price_vol / normal_price_vol - 1) * 100
    
    extreme_price_impact.loc[condition] = [
        extreme_price_avg,
        price_increase,
        vol_increase
    ]

print("Impact of Extreme Weather Conditions on Houston Hub Prices")
extreme_price_impact

# Create a bar chart comparing price increases during extreme conditions
plt.figure(figsize=(12, 6))
plt.bar(extreme_price_impact.index, extreme_price_impact['Price Increase (%)'])
plt.title('Price Increase During Extreme Weather Conditions (Houston)')
plt.xlabel('Weather Condition')
plt.ylabel('Price Increase (%)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Price Autocorrelation Analysis

Let's analyze the autocorrelation of electricity prices to identify patterns.

In [ ]:
# Calculate autocorrelation for Houston prices
from pandas.plotting import autocorrelation_plot

# Create autocorrelation plot for Houston hub
plt.figure(figsize=(12, 6))
autocorrelation_plot(price_data['HB_HOUSTON']['price'])
plt.title('Autocorrelation Plot for Houston Hub Prices')
plt.xlim(0, 168*2)  # Limit to 2 weeks (in hours)
plt.grid(True)
plt.show()

# Calculate and plot autocorrelation with confidence intervals
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ACF plot
plt.figure(figsize=(12, 6))
plot_acf(price_data['HB_HOUSTON']['price'], lags=168, alpha=0.05)  # One week of lags
plt.title('Autocorrelation Function for Houston Hub Prices')
plt.xlabel('Lag (hours)')
plt.grid(True)
plt.show()

# PACF plot
plt.figure(figsize=(12, 6))
plot_pacf(price_data['HB_HOUSTON']['price'], lags=48, alpha=0.05)  # Two days of lags
plt.title('Partial Autocorrelation Function for Houston Hub Prices')
plt.xlabel('Lag (hours)')
plt.grid(True)
plt.show()

## Cross-Correlation Between Weather and Price

Let's analyze the cross-correlation between weather variables and electricity prices.


In [ ]:
from scipy import signal

# Calculate cross-correlation between temperature and price
# First, align the series
aligned_data = align_time_series(price_data['HB_HOUSTON'], weather_data['Houston'])
price_series = aligned_data['price']
temp_series = aligned_data['temperature']

# Remove any NaN values
valid_data = aligned_data.dropna()
price_series = valid_data['price']
temp_series = valid_data['temperature']

# Calculate cross-correlation
max_lag = 168  # One week in hours
xcorr = signal.correlate(price_series - price_series.mean(), 
                         temp_series - temp_series.mean(), 
                         mode='full') / (len(price_series) * price_series.std() * temp_series.std())

# Calculate lag array
lags = np.arange(-max_lag, max_lag + 1)
xcorr = xcorr[len(xcorr)//2 - max_lag:len(xcorr)//2 + max_lag + 1]

# Plot cross-correlation
plt.figure(figsize=(12, 6))
plt.plot(lags, xcorr)
plt.title('Cross-Correlation: Temperature vs. Price (Houston)')
plt.xlabel('Lag (hours)')
plt.ylabel('Correlation Coefficient')
plt.grid(True)
plt.axhline(y=0, color='r', linestyle='-')

# Find the lag with maximum correlation
max_corr_lag = lags[np.argmax(np.abs(xcorr))]
plt.axvline(x=max_corr_lag, color='g', linestyle='--', 
            label=f'Max at lag={max_corr_lag} hours')
plt.legend()
plt.show()

## Price Return Analysis

Let's analyze price returns (percentage changes) to understand volatility patterns.a

In [ ]:
# Calculate hourly returns for all hubs
returns = pd.DataFrame()

for node in price_nodes:
    # Calculate percentage returns
    price_series = price_data[node]['price']
    returns[node] = price_series.pct_change() * 100  # Convert to percentage

# Remove extreme outliers for better visualization
for node in price_nodes:
    q1 = returns[node].quantile(0.01)
    q99 = returns[node].quantile(0.99)
    returns[node] = returns[node].clip(q1, q99)

# Plot return distributions
plt.figure(figsize=(12, 8))
for node in price_nodes:
    sns.kdeplot(returns[node].dropna(), label=node)

plt.title('Distribution of Hourly Price Returns')
plt.xlabel('Hourly Return (%)')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

# Calculate and display return statistics
return_stats = pd.DataFrame()

for node in price_nodes:
    node_stats = {
        'Mean Return (%)': returns[node].mean(),
        'Std Dev (%)': returns[node].std(),
        'Skewness': returns[node].skew(),
        'Kurtosis': returns[node].kurtosis(),
        'Positive Returns (%)': (returns[node] > 0).mean() * 100
    }
    return_stats[node] = pd.Series(node_stats)

print("Hourly Return Statistics Across ERCOT Hubs")
return_stats.T

## Conclusion

This exploratory data analysis has provided valuable insights into:

1. The statistical properties of ERCOT electricity prices
2. The frequency and magnitude of price spikes
3. Seasonal and temporal patterns in prices
4. Correlations between different ERCOT hubs
5. The relationship between weather variables and prices
6. Autocorrelation and cross-correlation patterns
7. Price return characteristics

These insights are essential for developing effective forecasting models and understanding the drivers of ERCOT electricity prices. 